  Gabriel Graells Solé - 205638
# Twitter Scrapper - 2020 USA Elections

The goal of this Notebook is to scrappe Tweets related to the election of USA on 2020. To do so we will do a stream search using Twitter's API and a set of keywords related to this topic. We will use **Tweepy** to easy the work with Tweeters API.

The elections ended the Tuesday, November 3, 2020.

---

Tweets by **keywords, location and date** : https://gist.github.com/nickhargreaves/44615385daf335166980

In [13]:
import json
import pandas as pd
import numpy as np
import datetime
import csv
from collections import Counter
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor

from google.colab import drive
drive.mount('/content/drive/')
PATH = '/content/drive/My Drive/Colab Notebooks/IR-WA_FinalProject/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Connect to the API

In [5]:
# Access token informations 
access_token1 = "1330533182681313288-yMxmNMMhN5pnDKzfRBRCK4XXcpeYvO"
access_token_secret1 = "mNOFxiv6JnjV4ze0EiuhXT4xSDkNAQ04NhKdB5g7uTycx"

consumer_key1 = "sDFQbByaPblJTMYaPzcX09sdv"
consumer_secret1 = "RoMLGdY3hgfyzZbj1U5x4BN5K8gIgk6lUmwJZA4o9ak0gotLlu"

In [6]:
# Connect to Twitter API
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

## Scappe Account

Some account can provide more trust-worthy information that others, for example news paper account should provide more usefull and trust worthy information than random users.

In [ ]:
accounts = ["@GuardianUS","@CNNPolitics","@nytimes","@washingtonpost"]
""" TO DO """

## Class Listener

**IMPORTANT**
Twitter has paramter **filter_level** which is used to rate relevance of tweets. **filter_level** takes values ["low", "medium", "high"]

In [7]:
class MyStreamListener(StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api, OUTPUT_FILENAME, stop_condition=10):
        """
        initialize the stream, with num. of tweets and saving the outputfile
        """
        
        # this line is needed to import the characteristics of the streaming API
        super(MyStreamListener, self).__init__()
        
        # to-count the number of tweets collected        
        self.num_tweets = 0
        
        # save filename
        self.filename = OUTPUT_FILENAME
        
        # stop-condition
        self.stop_condition = stop_condition
        

    def on_status(self, status):
        
        """
        this function runs each time a new bunch of tweets is retrived from the streaming
        """
        
        with open(self.filename, "a+") as f:
            tweet = status._json
            
            f.write(json.dumps(tweet) + '\n')
            #self.output.append(tweet)
            self.num_tweets += 1
        
            # Stop condition        
            if self.num_tweets <= self.stop_condition:
                return True
            else:
                return False
        

    def on_error(self, status):
        """
        function useful to handle errors. It's possible to personalize it 
        depending on the way we want to handle errors
        """
        
        print(status)
        #returning False in on_error disconnects the stream
        return False

In [55]:
%%time

OUTPUT_FILENAME = f"{PATH}2020_USA_elections.json"
stop_condition = 10

l = MyStreamListener(api, OUTPUT_FILENAME, stop_condition)
# here we recall the Stream Class from Tweepy to input the authentication info and our personalized listener 
stream = Stream(auth=api.auth, listener=l)


# keywords we may want decide to track 
TRACKING_KEYWORDS = ["#USAElections2020 ", "#uselections", "#USElections", 
                     "#2020election", "#2020Election","#USElectionResults",
                     "#vote","#voteblue", "#berniesanders","#donaldtrump"]
stream.filter(
    track=TRACKING_KEYWORDS, 
    async=False, 
    languages = ["en"],
   # filter_level = "high"
)

CPU times: user 32.1 ms, sys: 3.48 ms, total: 35.6 ms
Wall time: 41.2 s


## Data Visualisation

In [56]:
with open(f"{PATH}2020_USA_elections.json", "rb") as f:
    data = f.readlines()
    data = [json.loads(str_) for str_ in data]

In [61]:
df_tweets = pd.DataFrame.from_records(data)
df_tweets.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,extended_tweet,extended_entities,possibly_sensitive
0,Sun Nov 22 16:44:48 +0000 2020,1330552822484652034,1330552822484652034,RT @mansoortshams: America’s ISIS with assault...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,None,None,None,None,None,"{'id': 19239695, 'id_str': '19239695', 'name':...",None,None,None,None,{'created_at': 'Sun Nov 22 15:29:59 +0000 2020...,1.330503e+18,1330502557786124289,{'created_at': 'Sun Nov 22 13:25:03 +0000 2020...,"{'url': 'https://t.co/4kLUb0nC3G', 'expanded':...",True,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1606063488013,NaN,NaN,NaN
1,Sun Nov 22 16:44:53 +0000 2020,1330552845066788864,1330552845066788864,RT @theangiestanton: Some American Patriots de...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,None,"{'id': 253417802, 'id_str': '253417802', 'name...",None,None,None,None,{'created_at': 'Sun Nov 22 13:25:03 +0000 2020...,NaN,NaN,NaN,NaN,False,0,0,0,0,"{'hashtags': [{'text': '2020election', 'indice...",False,False,low,en,1606063493397,NaN,NaN,NaN
2,Sun Nov 22 16:44:53 +0000 2020,1330552847415521280,1330552847415521280,"Sorry, but if this is acceptable, then it is a...","<a href=""http://twitter.com/download/android"" ...",True,None,None,None,None,None,"{'id': 1028837748667547648, 'id_str': '1028837...",None,None,None,None,NaN,1.330503e+18,1330502557786124289,{'created_at': 'Sun Nov 22 13:25:03 +0000 2020...,"{'url': 'https://t.co/cC98BOPAN7', 'expanded':...",True,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,1606063493957,"{'full_text': 'Sorry, but if this is acceptabl...",NaN,NaN
3,Sun Nov 22 16:44:54 +0000 2020,1330552848350896129,1330552848350896129,RT @BetteMidler: The repulsive ugly vindictive...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,None,"{'id': 1240382543628775424, 'id_str': '1240382...",None,None,None,None,{'created_at': 'Sun Nov 22 15:41:23 +0000 2020...,NaN,NaN,NaN,NaN,False,0,0,0,0,"{'hashtags': [{'text': 'DonaldTrump', 'indices...",False,False,low,en,1606063494180,NaN,NaN,NaN
4,Sun Nov 22 16:45:00 +0000 2020,1330552874313584642,1330552874313584642,RT @theangiestanton: Some American Patriots de...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,None,"{'id': 861938461816877062, 'id_str': '86193846...",None,None,None,None,{'created_at': 'Sun Nov 22 13:25:03 +0000 2020...,NaN,NaN,NaN,NaN,False,0,0,0,0,"{'hashtags': [{'text': '2020election', 'indice...",False,False,low,en,1606063500370,NaN,NaN,NaN


With a simple visual inspection we can see that the dataset contains many column attributes that are non-relevant for our search engine. Thus, to reduce space use and consequently search time we will drop all unnecesary columns.

In [62]:
# source, timestamp_ms
drop_cols = ["id_str","truncated","in_reply_to_status_id","in_reply_to_status_id_str",
             "in_reply_to_user_id","in_reply_to_user_id_str",
             "in_reply_to_screen_name","geo","coordinates","place","contributors",
             "quoted_status_id","quoted_status_id_str","quoted_status","quote_count",
             "favorited","retweeted","lang","quoted_status_permalink","is_quote_status", 
             "extended_tweet","timestamp_ms","source"]

df_tweets = df_tweets.drop(drop_cols, axis = 1)
df_tweets.head()

,created_at,id,text,user,retweeted_status,reply_count,retweet_count,favorite_count,entities,filter_level,extended_entities,possibly_sensitive
0,Sun Nov 22 16:44:48 +0000 2020,1330552822484652034,RT @mansoortshams: America’s ISIS with assault...,"{'id': 19239695, 'id_str': '19239695', 'name':...",{'created_at': 'Sun Nov 22 15:29:59 +0000 2020...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",low,NaN,NaN
1,Sun Nov 22 16:44:53 +0000 2020,1330552845066788864,RT @theangiestanton: Some American Patriots de...,"{'id': 253417802, 'id_str': '253417802', 'name...",{'created_at': 'Sun Nov 22 13:25:03 +0000 2020...,0,0,0,"{'hashtags': [{'text': '2020election', 'indice...",low,NaN,NaN
2,Sun Nov 22 16:44:53 +0000 2020,1330552847415521280,"Sorry, but if this is acceptable, then it is a...","{'id': 1028837748667547648, 'id_str': '1028837...",NaN,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",low,NaN,NaN
3,Sun Nov 22 16:44:54 +0000 2020,1330552848350896129,RT @BetteMidler: The repulsive ugly vindictive...,"{'id': 1240382543628775424, 'id_str': '1240382...",{'created_at': 'Sun Nov 22 15:41:23 +0000 2020...,0,0,0,"{'hashtags': [{'text': 'DonaldTrump', 'indices...",low,NaN,NaN
4,Sun Nov 22 16:45:00 +0000 2020,1330552874313584642,RT @theangiestanton: Some American Patriots de...,"{'id': 861938461816877062, 'id_str': '86193846...",{'created_at': 'Sun Nov 22 13:25:03 +0000 2020...,0,0,0,"{'hashtags': [{'text': '2020election', 'indice...",low,NaN,NaN
